<a href="https://colab.research.google.com/github/junaid537/pytorch/blob/main/finalhw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
if not os.path.exists('/content/drive/MyDrive'):
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Google Drive is already mounted.")


Mounted at /content/drive


In [5]:
train_file = "/content/drive/MyDrive/dataHW4csci544/data/train"

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
train_file = "/content/drive/MyDrive/dataHW4csci544/data/train"
dev_file = "/content/drive/MyDrive/dataHW4csci544/data/dev"

In [ ]:

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from torch.nn.utils.rnn import pad_sequence
# import os

# # Paths
# train_file = "/content/drive/MyDrive/dataHW4csci544/data/train"
# dev_file = "/content/drive/MyDrive/dataHW4csci544/data/dev"
# test_file = "/content/drive/MyDrive/dataHW4csci544/data/test"
# output_dir = "/content/drive/MyDrive/dataHW4csci544/"

# # Optimized Hyperparameters
# EMBEDDING_DIM = 100
# HIDDEN_DIM = 256
# LSTM_LAYERS = 1
# DROPOUT = 0.2
# LINEAR_OUT_DIM = 128
# BATCH_SIZE = 32
# LEARNING_RATE = 0.01
# MOMENTUM = 0.9
# WEIGHT_DECAY = 5e-5
# EPOCHS = 30

# # Ensure output directory exists
# os.makedirs(output_dir, exist_ok=True)

# # Load dataset
# def load_data(filename, is_test=False):
#     sentences, labels = [], []
#     with open(filename, 'r', encoding='utf-8') as f:
#         sentence, label = [], []
#         for line in f:
#             line = line.strip()
#             if not line:
#                 if sentence:
#                     sentences.append(sentence)
#                     labels.append(label)
#                     sentence, label = [], []
#                 continue
#             parts = line.split()
#             if len(parts) < 2:
#                 continue
#             sentence.append(parts[1])
#             if not is_test:
#                 label.append(parts[2] if len(parts) > 2 else "O")
#             else:
#                 label.append("O")
#     if sentence:
#         sentences.append(sentence)
#         labels.append(label)
#     return sentences, labels

# # Load train, dev, and test data
# train_sentences, train_labels = load_data(train_file)
# dev_sentences, dev_labels = load_data(dev_file)
# test_sentences, _ = load_data(test_file, is_test=True)  # No labels for test set

# # Build vocabulary
# word_to_ix = {"<PAD>": 0, "<UNK>": 1}
# tag_to_ix = {}
# for sentence in train_sentences:
#     for word in sentence:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)

# for labels in train_labels:
#     for tag in labels:
#         if tag not in tag_to_ix:
#             tag_to_ix[tag] = len(tag_to_ix)

# ix_to_tag = {v: k for k, v in tag_to_ix.items()}

# # Dataset class
# class NERDataset(Dataset):
#     def __init__(self, sentences, labels, word_to_ix, tag_to_ix):
#         self.sentences = sentences
#         self.labels = labels
#         self.word_to_ix = word_to_ix
#         self.tag_to_ix = tag_to_ix

#     def __len__(self):
#         return len(self.sentences)

#     def __getitem__(self, idx):
#         sentence = self.sentences[idx]
#         label = self.labels[idx]
#         word_indices = [self.word_to_ix.get(w, 1) for w in sentence]
#         label_indices = [self.tag_to_ix.get(l, 0) for l in label]  # Default to 0 for unknown tags
#         return torch.tensor(word_indices), torch.tensor(label_indices)

# # Custom collate function
# def collate_fn(batch):
#     words, labels = zip(*batch)
#     words_padded = pad_sequence(words, batch_first=True, padding_value=0)
#     labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)
#     return words_padded, labels_padded

# # Create datasets and loaders
# train_dataset = NERDataset(train_sentences, train_labels, word_to_ix, tag_to_ix)
# dev_dataset = NERDataset(dev_sentences, dev_labels, word_to_ix, tag_to_ix)
# test_dataset = NERDataset(test_sentences, [["O"] * len(s) for s in test_sentences], word_to_ix, tag_to_ix)  # Dummy labels for test

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
# dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)  # FIXED: Now test_loader is defined!

# # Define BLSTM Model
# class BLSTM_NER(nn.Module):
#     def __init__(self, vocab_size, tagset_size):
#         super(BLSTM_NER, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, EMBEDDING_DIM)
#         self.lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, num_layers=LSTM_LAYERS,
#                             bidirectional=True, dropout=DROPOUT, batch_first=True)
#         self.fc = nn.Linear(HIDDEN_DIM * 2, LINEAR_OUT_DIM)
#         self.elu = nn.ELU()
#         self.classifier = nn.Linear(LINEAR_OUT_DIM, tagset_size)

#     def forward(self, x):
#         x = self.embedding(x)
#         x, _ = self.lstm(x)
#         x = self.fc(x)
#         x = self.elu(x)
#         x = self.classifier(x)
#         return x

# # Initialize model, loss, optimizer, and scheduler
# model = BLSTM_NER(len(word_to_ix), len(tag_to_ix))
# criterion = nn.CrossEntropyLoss(ignore_index=-1)

# optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, nesterov=True, weight_decay=WEIGHT_DECAY)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)

# # Training loop
# for epoch in range(EPOCHS):
#     model.train()
#     total_loss = 0

#     for words, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(words)

#         # Reshape for loss computation
#         outputs = outputs.view(-1, outputs.shape[-1])
#         labels = labels.view(-1).long()

#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()

#     scheduler.step()
#     print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, LR: {scheduler.get_last_lr()[0]}")

# # Save model
# torch.save(model.state_dict(), os.path.join(output_dir, "blstm1.pt"))
# print("Model saved!")

# # Evaluation function
# def evaluate(model, data_loader, sentences, output_file):
#     model.eval()
#     predictions = []
#     batch_offset = 0

#     with torch.no_grad():
#         for words, _ in data_loader:
#             outputs = model(words)
#             outputs = torch.argmax(outputs, dim=-1)

#             for batch_idx, preds in enumerate(outputs):
#                 sentence = sentences[batch_offset]
#                 for word_idx, pred in enumerate(preds[:len(sentence)]):
#                     predictions.append(f"{word_idx + 1} {sentence[word_idx]} {ix_to_tag[pred.item()]}\n")
#                 predictions.append("\n")
#                 batch_offset += 1

#     with open(output_file, "w", encoding="utf-8") as f:
#         f.writelines(predictions)

#     print(f"Saved predictions to {output_file}")

# # Evaluate on dev set
# evaluate(model, dev_loader, dev_sentences, os.path.join(output_dir, "dev.out"))

# # Evaluate on test set
# evaluate(model, test_loader, test_sentences, os.path.join(output_dir, "test.out"))

# print("Training completed. Model and outputs saved!")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import os

# Paths
train_file = "/content/drive/MyDrive/dataHW4csci544/data/train"
dev_file = "/content/drive/MyDrive/dataHW4csci544/data/dev"
test_file = "/content/drive/MyDrive/dataHW4csci544/data/test"
output_dir = "/content/drive/MyDrive/dataHW4csci544/"

# Optimized Hyperparameters
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
LSTM_LAYERS = 1
DROPOUT = 0.12  # Lower dropout to improve recall
LINEAR_OUT_DIM = 128
BATCH_SIZE = 32
LEARNING_RATE = 0.01  # More stable learning
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-5  # Less regularization to prevent underfitting
EPOCHS = 35  # Train longer for better F1
# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load dataset
def load_data(filename, is_test=False):
    sentences, labels = [], []
    with open(filename, 'r', encoding='utf-8') as f:
        sentence, label = [], []
        for line in f:
            line = line.strip()
            if not line:
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                    sentence, label = [], []
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            sentence.append(parts[1])
            if not is_test:
                label.append(parts[2] if len(parts) > 2 else "O")
            else:
                label.append("O")
    if sentence:
        sentences.append(sentence)
        labels.append(label)
    return sentences, labels

# Load train, dev, and test data
train_sentences, train_labels = load_data(train_file)
dev_sentences, dev_labels = load_data(dev_file)
test_sentences, _ = load_data(test_file, is_test=True)  # No labels for test set

# Build vocabulary
word_to_ix = {"<PAD>": 0, "<UNK>": 1}
tag_to_ix = {}
for sentence in train_sentences:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

for labels in train_labels:
    for tag in labels:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

ix_to_tag = {v: k for k, v in tag_to_ix.items()}

# Dataset class
class NERDataset(Dataset):
    def __init__(self, sentences, labels, word_to_ix, tag_to_ix):
        self.sentences = sentences
        self.labels = labels
        self.word_to_ix = word_to_ix
        self.tag_to_ix = tag_to_ix

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]
        word_indices = [self.word_to_ix.get(w, 1) for w in sentence]
        label_indices = [self.tag_to_ix.get(l, 0) for l in label]  # Default to 0 for unknown tags
        return torch.tensor(word_indices), torch.tensor(label_indices)

# Custom collate function
def collate_fn(batch):
    words, labels = zip(*batch)
    words_padded = pad_sequence(words, batch_first=True, padding_value=0)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)
    return words_padded, labels_padded

# Create datasets and loaders
train_dataset = NERDataset(train_sentences, train_labels, word_to_ix, tag_to_ix)
dev_dataset = NERDataset(dev_sentences, dev_labels, word_to_ix, tag_to_ix)
test_dataset = NERDataset(test_sentences, [["O"] * len(s) for s in test_sentences], word_to_ix, tag_to_ix)  # Dummy labels for test

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Define BLSTM Model
class BLSTM_NER(nn.Module):
    def __init__(self, vocab_size, tagset_size):
        super(BLSTM_NER, self).__init__()
        self.embedding = nn.Embedding(vocab_size, EMBEDDING_DIM)
        self.lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, num_layers=LSTM_LAYERS,
                            bidirectional=True, dropout=DROPOUT, batch_first=True)
        self.fc = nn.Linear(HIDDEN_DIM * 2, LINEAR_OUT_DIM)
        self.elu = nn.ELU()
        self.classifier = nn.Linear(LINEAR_OUT_DIM, tagset_size)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x)
        x = self.elu(x)
        x = self.classifier(x)
        return x

# Initialize model, loss, optimizer, and scheduler
model = BLSTM_NER(len(word_to_ix), len(tag_to_ix))
criterion = nn.CrossEntropyLoss(ignore_index=-1)

optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, nesterov=True, weight_decay=WEIGHT_DECAY)

# ✅ Learning Rate Scheduler (Reduce LR every 10 epochs instead of 5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for words, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(words)

        # Reshape for loss computation
        outputs = outputs.view(-1, outputs.shape[-1])
        labels = labels.view(-1).long()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, LR: {scheduler.get_last_lr()[0]}")

# Save model
torch.save(model.state_dict(), os.path.join(output_dir, "blstm1.pt"))
print("Model saved!")

# Evaluation function
def evaluate(model, data_loader, sentences, output_file):
    model.eval()
    predictions = []
    batch_offset = 0

    with torch.no_grad():
        for words, _ in data_loader:
            outputs = model(words)
            outputs = torch.argmax(outputs, dim=-1)

            for batch_idx, preds in enumerate(outputs):
                sentence = sentences[batch_offset]
                for word_idx, pred in enumerate(preds[:len(sentence)]):
                    predictions.append(f"{word_idx + 1} {sentence[word_idx]} {ix_to_tag[pred.item()]}\n")
                predictions.append("\n")
                batch_offset += 1

    with open(output_file, "w", encoding="utf-8") as f:
        f.writelines(predictions)

    print(f"Saved predictions to {output_file}")

# Evaluate on dev set
evaluate(model, dev_loader, dev_sentences, os.path.join(output_dir, "dev.out"))

# Evaluate on test set
evaluate(model, test_loader, test_sentences, os.path.join(output_dir, "test.out"))

print("Training completed. Model and outputs saved!")


Epoch 1, Loss: 364.3246, LR: 0.01
Epoch 2, Loss: 316.1281, LR: 0.01
Epoch 3, Loss: 290.7997, LR: 0.01
Epoch 4, Loss: 261.6946, LR: 0.01
Epoch 5, Loss: 232.1846, LR: 0.01
Epoch 6, Loss: 209.5106, LR: 0.01
Epoch 7, Loss: 192.0653, LR: 0.01
Epoch 8, Loss: 177.5068, LR: 0.01
Epoch 9, Loss: 164.7305, LR: 0.01
Epoch 10, Loss: 152.8905, LR: 0.01
Epoch 11, Loss: 142.2381, LR: 0.01
Epoch 12, Loss: 132.0019, LR: 0.01
Epoch 13, Loss: 122.4524, LR: 0.01
Epoch 14, Loss: 113.9070, LR: 0.01
Epoch 15, Loss: 105.0443, LR: 0.008
Epoch 16, Loss: 96.6448, LR: 0.008
Epoch 17, Loss: 89.9495, LR: 0.008
Epoch 18, Loss: 84.2575, LR: 0.008
Epoch 19, Loss: 78.6785, LR: 0.008
Epoch 20, Loss: 73.1632, LR: 0.008
Epoch 21, Loss: 67.9451, LR: 0.008
Epoch 22, Loss: 63.3880, LR: 0.008
Epoch 23, Loss: 58.5493, LR: 0.008
Epoch 24, Loss: 54.2120, LR: 0.008
Epoch 25, Loss: 50.1985, LR: 0.008
Epoch 26, Loss: 46.4349, LR: 0.008
Epoch 27, Loss: 43.1419, LR: 0.008
Epoch 28, Loss: 39.6696, LR: 0.008
Epoch 29, Loss: 36.8567, LR:

In [ ]:
#Code with 70 F1
#using SGD
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from torch.nn.utils.rnn import pad_sequence
# import os

# # Paths
# train_file = "/content/drive/MyDrive/dataHW4csci544/data/train"
# dev_file = "/content/drive/MyDrive/dataHW4csci544/data/dev"
# test_file = "/content/drive/MyDrive/dataHW4csci544/data/test"
# output_dir = "/content/drive/MyDrive/dataHW4csci544/"

# # Hyperparameters
# # EMBEDDING_DIM = 100
# # HIDDEN_DIM = 256
# # LSTM_LAYERS = 1
# # DROPOUT = 0.33
# # LINEAR_OUT_DIM = 128
# # BATCH_SIZE = 32
# # LEARNING_RATE = 0.1  # Adjusted for SGD
# # MOMENTUM = 0.9  # Momentum for SGD
# # WEIGHT_DECAY = 1e-4  # L2 Regularization
# # EPOCHS = 10  # Increased for better training
# EMBEDDING_DIM = 100
# HIDDEN_DIM = 256
# LSTM_LAYERS = 1
# DROPOUT = 0.2
# LINEAR_OUT_DIM = 128
# BATCH_SIZE = 32
# LEARNING_RATE = 0.01  # Adjusted for SGD
# MOMENTUM = 0.9  # Momentum for SGD
# WEIGHT_DECAY = 5e-5  # L2 Regularization
# EPOCHS = 20  # Increased for better training
# # Ensure output directory exists
# os.makedirs(output_dir, exist_ok=True)

# # Load dataset (Handles both train/dev and test files)
# def load_data(filename, is_test=False):
#     sentences, labels = [], []
#     with open(filename, 'r', encoding='utf-8') as f:
#         sentence, label = [], []
#         for line in f:
#             line = line.strip()
#             if not line:
#                 if sentence:
#                     sentences.append(sentence)
#                     labels.append(label)
#                     sentence, label = [], []
#                 continue
#             parts = line.split()
#             if len(parts) < 2:
#                 continue  # Skip malformed lines
#             sentence.append(parts[1])  # Word
#             if not is_test:
#                 label.append(parts[2] if len(parts) > 2 else "O")  # NER Tag
#             else:
#                 label.append("O")  # Assign "O" for test set
#     if sentence:
#         sentences.append(sentence)
#         labels.append(label)
#     return sentences, labels

# # Load train and dev data
# train_sentences, train_labels = load_data(train_file)
# dev_sentences, dev_labels = load_data(dev_file)

# # Build vocabulary
# word_to_ix = {"<PAD>": 0, "<UNK>": 1}
# tag_to_ix = {}
# for sentence in train_sentences:
#     for word in sentence:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)

# for labels in train_labels:
#     for tag in labels:
#         if tag not in tag_to_ix:
#             tag_to_ix[tag] = len(tag_to_ix)

# ix_to_tag = {v: k for k, v in tag_to_ix.items()}

# # Dataset class
# class NERDataset(Dataset):
#     def __init__(self, sentences, labels, word_to_ix, tag_to_ix):
#         self.sentences = sentences
#         self.labels = labels
#         self.word_to_ix = word_to_ix
#         self.tag_to_ix = tag_to_ix

#     def __len__(self):
#         return len(self.sentences)

#     def __getitem__(self, idx):
#         sentence = self.sentences[idx]
#         label = self.labels[idx]
#         word_indices = [self.word_to_ix.get(w, 1) for w in sentence]
#         label_indices = [self.tag_to_ix[l] for l in label]
#         return torch.tensor(word_indices), torch.tensor(label_indices)

# # Custom collate function for dynamic padding
# def collate_fn(batch):
#     words, labels = zip(*batch)
#     words_padded = pad_sequence(words, batch_first=True, padding_value=0)
#     labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)
#     return words_padded, labels_padded

# # Create datasets and loaders
# train_dataset = NERDataset(train_sentences, train_labels, word_to_ix, tag_to_ix)
# dev_dataset = NERDataset(dev_sentences, dev_labels, word_to_ix, tag_to_ix)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
# dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# # Define BLSTM Model
# class BLSTM_NER(nn.Module):
#     def __init__(self, vocab_size, tagset_size):
#         super(BLSTM_NER, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, EMBEDDING_DIM)
#         self.lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, num_layers=LSTM_LAYERS,
#                             bidirectional=True, dropout=DROPOUT, batch_first=True)
#         self.fc = nn.Linear(HIDDEN_DIM * 2, LINEAR_OUT_DIM)
#         self.elu = nn.ELU()
#         self.classifier = nn.Linear(LINEAR_OUT_DIM, tagset_size)

#     def forward(self, x):
#         x = self.embedding(x)
#         x, _ = self.lstm(x)
#         x = self.fc(x)
#         x = self.elu(x)
#         x = self.classifier(x)
#         return x

# # Initialize model, loss, and optimizer (Using SGD Now)
# model = BLSTM_NER(len(word_to_ix), len(tag_to_ix))
# criterion = nn.CrossEntropyLoss(ignore_index=-1)

# # **Using SGD Instead of Adam**
# #optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
# optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, nesterov=True, weight_decay=WEIGHT_DECAY)
# # Learning Rate Scheduler (Reduce LR every 5 epochs)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)
# # Training loop (Fixed)
# for epoch in range(EPOCHS):
#     model.train()
#     total_loss = 0

#     for words, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(words)  # [batch_size, max_seq_len, num_classes]

#         # Reshape for loss computation
#         outputs = outputs.view(-1, outputs.shape[-1])  # [batch_size * max_seq_len, num_classes]
#         labels = labels.view(-1).long()  # [batch_size * max_seq_len]

#         # Ensure loss ignores padding labels (-1)
#         loss = criterion(outputs, labels)

#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()

#     print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# # Save model
# torch.save(model.state_dict(), os.path.join(output_dir, "blstm1.pt"))
# print("Model saved!")

# # Evaluation function (Fixed Indexing)
# def evaluate(model, data_loader, sentences, output_file):
#     model.eval()
#     predictions = []
#     batch_offset = 0  # Ensure correct sentence order

#     with torch.no_grad():
#         for words, _ in data_loader:
#             outputs = model(words)
#             outputs = torch.argmax(outputs, dim=-1)

#             for batch_idx, preds in enumerate(outputs):
#                 sentence = sentences[batch_offset]
#                 for word_idx, pred in enumerate(preds[:len(sentence)]):
#                     predictions.append(f"{word_idx + 1} {sentence[word_idx]} {ix_to_tag[pred.item()]}\n")
#                 predictions.append("\n")  # Ensure sentence separation
#                 batch_offset += 1  # Move to next sentence

#     with open(output_file, "w", encoding="utf-8") as f:
#         f.writelines(predictions)

#     print(f"Saved predictions to {output_file}")

# # Evaluate on dev set
# evaluate(model, dev_loader, dev_sentences, os.path.join(output_dir, "dev.out"))

# # Evaluate on test set
# test_sentences, test_labels = load_data(test_file, is_test=True)
# test_dataset = NERDataset(test_sentences, test_labels, word_to_ix, tag_to_ix)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
# evaluate(model, test_loader, test_sentences, os.path.join(output_dir, "test.out"))

# print("Training completed. Model and outputs saved!")


In [ ]:
# USing Adams
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from torch.nn.utils.rnn import pad_sequence
# import os

# # Paths
# train_file = "/content/drive/MyDrive/dataHW4csci544/data/train"
# dev_file = "/content/drive/MyDrive/dataHW4csci544/data/dev"
# test_file = "/content/drive/MyDrive/dataHW4csci544/data/test"
# output_dir = "/content/drive/MyDrive/dataHW4csci544/"

# # Hyperparameters
# EMBEDDING_DIM = 100
# HIDDEN_DIM = 256
# LSTM_LAYERS = 1
# DROPOUT = 0.33
# LINEAR_OUT_DIM = 128
# BATCH_SIZE = 32
# LEARNING_RATE = 0.001  # Reduced for better stability
# EPOCHS = 5  # Increased for better training

# # Ensure output directory exists
# os.makedirs(output_dir, exist_ok=True)

# # Load dataset (Handles both train/dev and test files)
# def load_data(filename, is_test=False):
#     sentences, labels = [], []
#     with open(filename, 'r', encoding='utf-8') as f:
#         sentence, label = [], []
#         for line in f:
#             line = line.strip()
#             if not line:
#                 if sentence:
#                     sentences.append(sentence)
#                     labels.append(label)
#                     sentence, label = [], []
#                 continue
#             parts = line.split()
#             if len(parts) < 2:
#                 continue  # Skip malformed lines
#             sentence.append(parts[1])  # Word
#             if not is_test:
#                 label.append(parts[2] if len(parts) > 2 else "O")  # NER Tag
#             else:
#                 label.append("O")  # Assign "O" for test set
#     if sentence:
#         sentences.append(sentence)
#         labels.append(label)
#     return sentences, labels

# # Load train and dev data
# train_sentences, train_labels = load_data(train_file)
# dev_sentences, dev_labels = load_data(dev_file)

# # Build vocabulary
# word_to_ix = {"<PAD>": 0, "<UNK>": 1}
# tag_to_ix = {}
# for sentence in train_sentences:
#     for word in sentence:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)

# for labels in train_labels:
#     for tag in labels:
#         if tag not in tag_to_ix:
#             tag_to_ix[tag] = len(tag_to_ix)

# ix_to_tag = {v: k for k, v in tag_to_ix.items()}

# # Dataset class
# class NERDataset(Dataset):
#     def __init__(self, sentences, labels, word_to_ix, tag_to_ix):
#         self.sentences = sentences
#         self.labels = labels
#         self.word_to_ix = word_to_ix
#         self.tag_to_ix = tag_to_ix

#     def __len__(self):
#         return len(self.sentences)

#     def __getitem__(self, idx):
#         sentence = self.sentences[idx]
#         label = self.labels[idx]
#         word_indices = [self.word_to_ix.get(w, 1) for w in sentence]
#         label_indices = [self.tag_to_ix[l] for l in label]
#         return torch.tensor(word_indices), torch.tensor(label_indices)

# # Custom collate function for dynamic padding
# def collate_fn(batch):
#     words, labels = zip(*batch)
#     words_padded = pad_sequence(words, batch_first=True, padding_value=0)
#     labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)
#     return words_padded, labels_padded

# # Create datasets and loaders
# train_dataset = NERDataset(train_sentences, train_labels, word_to_ix, tag_to_ix)
# dev_dataset = NERDataset(dev_sentences, dev_labels, word_to_ix, tag_to_ix)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
# dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# # Define BLSTM Model
# class BLSTM_NER(nn.Module):
#     def __init__(self, vocab_size, tagset_size):
#         super(BLSTM_NER, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, EMBEDDING_DIM)
#         self.lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, num_layers=LSTM_LAYERS,
#                             bidirectional=True, dropout=DROPOUT, batch_first=True)
#         self.fc = nn.Linear(HIDDEN_DIM * 2, LINEAR_OUT_DIM)
#         self.elu = nn.ELU()
#         self.classifier = nn.Linear(LINEAR_OUT_DIM, tagset_size)

#     def forward(self, x):
#         x = self.embedding(x)
#         x, _ = self.lstm(x)
#         x = self.fc(x)
#         x = self.elu(x)
#         x = self.classifier(x)
#         return x

# # Initialize model, loss, and optimizer
# model = BLSTM_NER(len(word_to_ix), len(tag_to_ix))
# criterion = nn.CrossEntropyLoss(ignore_index=-1)
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)  # Switched to Adam for better convergence

# # Training loop (Fixed)
# for epoch in range(EPOCHS):
#     model.train()
#     total_loss = 0

#     for words, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(words)  # [batch_size, max_seq_len, num_classes]

#         # Reshape for loss computation
#         outputs = outputs.view(-1, outputs.shape[-1])  # [batch_size * max_seq_len, num_classes]
#         labels = labels.view(-1).long()  # [batch_size * max_seq_len]

#         # Ensure loss ignores padding labels (-1)
#         loss = criterion(outputs, labels)

#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()

#     print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# # Save model
# torch.save(model.state_dict(), os.path.join(output_dir, "blstm1.pt"))
# print("Model saved!")

# # Evaluation function (Fixed Indexing)
# def evaluate(model, data_loader, sentences, output_file):
#     model.eval()
#     predictions = []
#     batch_offset = 0  # Ensure correct sentence order

#     with torch.no_grad():
#         for words, _ in data_loader:
#             outputs = model(words)
#             outputs = torch.argmax(outputs, dim=-1)

#             for batch_idx, preds in enumerate(outputs):
#                 sentence = sentences[batch_offset]
#                 for word_idx, pred in enumerate(preds[:len(sentence)]):
#                     predictions.append(f"{word_idx + 1} {sentence[word_idx]} {ix_to_tag[pred.item()]}\n")  # Start from 1
#                 predictions.append("\n")  # Ensure sentence separation
#                 batch_offset += 1  # Move to next sentence

#     with open(output_file, "w", encoding="utf-8") as f:
#         f.writelines(predictions)

#     print(f"Saved predictions to {output_file}")


# # Evaluate on dev set
# evaluate(model, dev_loader, dev_sentences, os.path.join(output_dir, "dev.out"))

# # Load and process test data
# test_sentences, test_labels = load_data(test_file, is_test=True)
# test_dataset = NERDataset(test_sentences, test_labels, word_to_ix, tag_to_ix)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# # Evaluate on test set
# evaluate(model, test_loader, test_sentences, os.path.join(output_dir, "test.out"))

# print("Training completed. Model and outputs saved!")


In [ ]:
!python /content/drive/MyDrive/dataHW4csci544/eval.py -p /content/drive/MyDrive/dataHW4csci544/dev.out -g /content/drive/MyDrive/dataHW4csci544/data/dev


processed 51578 tokens with 5942 phrases; found: 4804 phrases; correct: 3473.
accuracy:  92.94%; precision:  72.29%; recall:  58.45%; FB1:  64.64
              LOC: precision:  83.68%; recall:  76.21%; FB1:  79.77  1673
             MISC: precision:  69.60%; recall:  60.85%; FB1:  64.93  806
              ORG: precision:  68.95%; recall:  54.81%; FB1:  61.07  1066
              PER: precision:  61.72%; recall:  42.18%; FB1:  50.11  1259


In [ ]:
python eval.py -p /content/drive/MyDrive/dataHW4csci544/dev.out -g /content/drive/MyDrive/dataHW4csci544/data/dev


In [ ]:
#working
# import torch
# from torch.utils.data import Dataset, DataLoader
# import torch.nn as nn
# import torch.optim as optim
# from torch.nn.utils.rnn import pad_sequence
# import os

# # Define the dataset class
# class NERDataset(Dataset):
#     def __init__(self, file_path, word_to_idx, label_to_idx):
#         self.word_to_idx = word_to_idx
#         self.label_to_idx = label_to_idx
#         self.sentences, self.labels = self.load_data(file_path)

#     def load_data(self, file_path):
#         sentences = []
#         labels = []

#         with open(file_path, 'r') as file:
#             sentence = []
#             label = []

#             for line in file:
#                 line = line.strip()
#                 if line == "":
#                     if sentence:
#                         sentences.append(sentence)
#                         labels.append(label)
#                     sentence = []
#                     label = []
#                 else:
#                     parts = line.split()
#                     if len(parts) == 3:
#                         idx, word, gold_label = parts
#                     elif len(parts) == 2:
#                         idx, word = parts
#                         gold_label = "O"  # Default label if missing
#                     else:
#                         continue

#                     sentence.append(self.word_to_idx.get(word, self.word_to_idx["<UNK>"]))
#                     label.append(self.label_to_idx.get(gold_label, self.label_to_idx["O"]))

#         return sentences, labels

#     def __len__(self):
#         return len(self.sentences)

#     def __getitem__(self, idx):
#         return torch.tensor(self.sentences[idx]), torch.tensor(self.labels[idx])

# # Collate function to pad sequences
# def collate_fn(batch):
#     sentences, labels = zip(*batch)
#     padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=0)
#     padded_labels = pad_sequence(labels, batch_first=True, padding_value=0)
#     return padded_sentences, padded_labels

# # Define the BLSTM model
# class BLSTMModel(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
#         super(BLSTMModel, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
#         self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True, dropout=0.33)
#         self.fc = nn.Linear(hidden_size * 2, 128)
#         self.elu = nn.ELU()
#         self.classifier = nn.Linear(128, output_size)

#     def forward(self, x):
#         x = self.embedding(x)
#         lstm_out, _ = self.lstm(x)
#         linear_out = self.fc(lstm_out)
#         elu_out = self.elu(linear_out)
#         logits = self.classifier(elu_out)
#         return logits

# # Training function
# def train_model(train_loader, model, criterion, optimizer, num_epochs=5):
#     model.train()
#     for epoch in range(num_epochs):
#         total_loss = 0
#         for sentences, labels in train_loader:
#             optimizer.zero_grad()
#             outputs = model(sentences)
#             loss = criterion(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
#             total_loss += loss.item()
#             loss.backward()
#             optimizer.step()
#         print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")

# # Save predictions in conll03eval format
# def save_predictions(predictions, data_file, output_file):
#     with open(data_file, 'r') as f, open(output_file, 'w') as out_f:
#         i = 0
#         predictions = [pred.item() for batch in predictions for pred in batch.flatten()]
#         for line in f:
#             line = line.strip()
#             if line:
#                 parts = line.split()
#                 if len(parts) >= 2:
#                     idx, word = parts[:2]
#                     pred = predictions[i]
#                     out_f.write(f"{idx} {word} {pred}\n")
#                     i += 1
#             else:
#                 out_f.write("\n")

# # Make predictions
# def make_predictions(data_loader, model):
#     model.eval()
#     predictions = []
#     with torch.no_grad():
#         for sentences, _ in data_loader:
#             outputs = model(sentences)
#             _, predicted_labels = torch.max(outputs, -1)
#             predictions.extend(predicted_labels)
#     return predictions

# # Prepare word and label mappings
# word_to_idx = {"<PAD>": 0, "<UNK>": 1}  # Special tokens
# label_to_idx = {}

# # Load training data to populate mappings
# train_data_path = "/content/drive/MyDrive/dataHW4csci544/data/train"
# with open(train_data_path, 'r') as f:
#     for line in f:
#         line = line.strip()
#         if line:
#             _, word, label = line.split()
#             if word not in word_to_idx:
#                 word_to_idx[word] = len(word_to_idx)
#             if label not in label_to_idx:
#                 label_to_idx[label] = len(label_to_idx)

# idx_to_label = {idx: label for label, idx in label_to_idx.items()}  # Reverse mapping

# # Load datasets
# train_data = NERDataset(train_data_path, word_to_idx, label_to_idx)
# train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_fn)

# # Model setup
# input_size = len(word_to_idx)
# embedding_dim = 100
# hidden_size = 256
# output_size = len(label_to_idx)
# model = BLSTMModel(input_size, embedding_dim, hidden_size, output_size)

# # Loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# # Train the model
# train_model(train_loader, model, criterion, optimizer, num_epochs=5)

# # Save the trained model
# model_save_path = "/content/drive/MyDrive/dataHW4csci544/blstm1.pt"
# torch.save(model.state_dict(), model_save_path)
# print(f"Model saved to {model_save_path}")

# # Evaluate on Dev and Test Sets
# dev_data_path = "/content/drive/MyDrive/dataHW4csci544/data/dev"
# test_data_path = "/content/drive/MyDrive/dataHW4csci544/data/test"

# dev_data = NERDataset(dev_data_path, word_to_idx, label_to_idx)
# test_data = NERDataset(test_data_path, word_to_idx, label_to_idx)

# dev_loader = DataLoader(dev_data, batch_size=32, shuffle=False, collate_fn=collate_fn)
# test_loader = DataLoader(test_data, batch_size=32, shuffle=False, collate_fn=collate_fn)

# # Get predictions
# dev_predictions = make_predictions(dev_loader, model)
# test_predictions = make_predictions(test_loader, model)

# # Save predictions
# save_predictions(dev_predictions, dev_data_path, "/content/drive/MyDrive/dataHW4csci544/dev.out")
# save_predictions(test_predictions, test_data_path, "/content/drive/MyDrive/dataHW4csci544/test.out")
# print("Predictions saved.")



In [ ]:
# # Evaluate using conll03eval
# eval_command = "python /content/drive/MyDrive/dataHW4csci544/eval.py -p /content/drive/MyDrive/dataHW4csci544/dev.out -g /content/drive/MyDrive/dataHW4csci544/data/dev"
# os.system(eval_command)


In [ ]:
# !python3 '/content/drive/MyDrive/dataHW4csci544/eval.py' -p '/content/drive/MyDrive/dataHW4csci544/dev.out' -g '/content/drive/MyDrive/dataHW4csci544/data/dev'



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!python /content/drive/MyDrive/dataHW4csci544/eval.py -p /content/drive/MyDrive/dataHW4csci544/dev.out -g /content/drive/MyDrive/dataHW4csci544/data/dev


In [ ]:
!ls '/content/drive/MyDrive/dataHW4csci544/'


In [ ]:
#working
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from torch.nn.utils.rnn import pad_sequence
# import numpy as np

# # Hyperparameters
# EMBEDDING_DIM = 100
# HIDDEN_DIM = 256
# DROPOUT = 0.33
# LINEAR_OUT_DIM = 128
# NUM_LSTM_LAYERS = 1
# BATCH_SIZE = 32  # To be tuned
# LEARNING_RATE = 0.01  # To be tuned
# NUM_EPOCHS = 10  # To be tuned

# # Load Vocabulary and Labels
# def build_vocab_and_labels(file_path):
#     word_vocab = {"<PAD>": 0, "<UNK>": 1}
#     label_vocab = {}

#     with open(file_path, 'r') as f:
#         for line in f:
#             line = line.strip()
#             if line:
#                 _, word, label = line.split()
#                 if word not in word_vocab:
#                     word_vocab[word] = len(word_vocab)
#                 if label not in label_vocab:
#                     label_vocab[label] = len(label_vocab)

#     return word_vocab, label_vocab

# # Dataset Class
# class NERDataset(Dataset):
#     def __init__(self, file_path, word_vocab, label_vocab):
#         self.sentences, self.labels = self.load_data(file_path, word_vocab, label_vocab)

#     def load_data(self, file_path, word_vocab, label_vocab):
#         sentences, labels = [], []
#         sentence, label = [], []

#         with open(file_path, 'r') as f:
#             for line in f:
#                 line = line.strip()
#                 if line == "":
#                     if sentence:
#                         sentences.append(sentence)
#                         labels.append(label)
#                     sentence, label = [], []
#                 else:
#                     _, word, ner_tag = line.split()
#                     sentence.append(word_vocab.get(word, word_vocab["<UNK>"]))
#                     label.append(label_vocab[ner_tag])

#         return sentences, labels

#     def __len__(self):
#         return len(self.sentences)

#     def __getitem__(self, idx):
#         return torch.tensor(self.sentences[idx]), torch.tensor(self.labels[idx])

# # Collate function for batching
# def collate_fn(batch):
#     sentences, labels = zip(*batch)
#     padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=0)
#     padded_labels = pad_sequence(labels, batch_first=True, padding_value=0)
#     return padded_sentences, padded_labels

# # BLSTM Model
# class BLSTMModel(nn.Module):
#     def __init__(self, vocab_size, num_labels):
#         super(BLSTMModel, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, EMBEDDING_DIM, padding_idx=0)
#         self.lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, NUM_LSTM_LAYERS,
#                             batch_first=True, bidirectional=True, dropout=DROPOUT)
#         self.fc = nn.Linear(HIDDEN_DIM * 2, LINEAR_OUT_DIM)
#         self.elu = nn.ELU()
#         self.classifier = nn.Linear(LINEAR_OUT_DIM, num_labels)

#     def forward(self, x):
#         x = self.embedding(x)
#         lstm_out, _ = self.lstm(x)
#         linear_out = self.fc(lstm_out)
#         activated = self.elu(linear_out)
#         output = self.classifier(activated)
#         return output

# # Training Function
# def train_model(model, train_loader, criterion, optimizer):
#     model.train()
#     total_loss = 0
#     for sentences, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(sentences)
#         loss = criterion(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     return total_loss / len(train_loader)

# # Evaluation Function
# def evaluate_model(model, data_loader):
#     model.eval()
#     predictions = []
#     with torch.no_grad():
#         for sentences, _ in data_loader:
#             outputs = model(sentences)
#             _, predicted_labels = torch.max(outputs, -1)
#             predictions.append(predicted_labels)
#     return predictions

# # Load Data

# word_vocab, label_vocab = build_vocab_and_labels(train_file)

# train_data = NERDataset(train_file, word_vocab, label_vocab)
# dev_data = NERDataset(dev_file, word_vocab, label_vocab)

# train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
# dev_loader = DataLoader(dev_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# # Initialize Model
# num_labels = len(label_vocab)
# model = BLSTMModel(len(word_vocab), num_labels)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

# # Train the Model
# for epoch in range(NUM_EPOCHS):
#     loss = train_model(model, train_loader, criterion, optimizer)
#     print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {loss:.4f}")

# # Save Model
# torch.save(model.state_dict(), "blstm_ner.pt")

# # Make Predictions
# dev_predictions = evaluate_model(model, dev_loader)

# # Save Predictions in conll03eval format
# import torch

# def save_predictions(predictions, data_file, output_file):
#     with open(data_file, 'r') as f, open(output_file, 'w') as out_f:
#         i = 0
#         # Flatten predictions to a single list
#         predictions = torch.cat(predictions).tolist()  # Convert tensor batches into a single list

#         for line in f:
#             line = line.strip()
#             if line:  # Not an empty line
#                 idx, word, gold = line.split()
#                 pred = predictions[i]  # Extract prediction correctly
#                 out_f.write(f"{idx} {word} {pred}\n")
#                 i += 1
#             else:
#                 out_f.write("\n")  # Separate sentences properly


# # Run Evaluation
# import os
# os.system("python eval.py -p dev_predictions.out -g dev.txt")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_file = "/content/drive/MyDrive/dataHW4csci544/data/train"
dev_file = "/content/drive/MyDrive/dataHW4csci544/data/dev"

In [ ]:
#python eval.py -p /content/drive/MyDrive/dataHW4csci544/dev1.out -g /content/drive/MyDrive/dataHW4csci544/data/dev


## Task 2

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import os
import numpy as np
import gzip

# Paths
train_file = "/content/drive/MyDrive/hw4csci544/data/train"
dev_file = "/content/drive/MyDrive/hw4csci544/data/dev"
test_file = "/content/drive/MyDrive/hw4csci544/data/test"
glove_path = "/content/drive/MyDrive/hw4csci544/glove.6B.100d.gz"  # GloVe embeddings file
output_dir = "/content/drive/MyDrive/hw4csci544/"

# Hyperparameters
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
LSTM_LAYERS = 1
DROPOUT = 0.1
LINEAR_OUT_DIM = 128
BATCH_SIZE = 32
LEARNING_RATE = 0.01
MOMENTUM = 0.85
WEIGHT_DECAY = 3e-5
EPOCHS = 35

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Extract GloVe embeddings from .gz file
def load_glove_embeddings(glove_path, embedding_dim=100):
    glove_embeddings = {}
    with gzip.open(glove_path, "rt", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            word = parts[0]
            vector = np.array(parts[1:], dtype=np.float32)
            glove_embeddings[word] = vector
    return glove_embeddings

glove_embeddings = load_glove_embeddings(glove_path, EMBEDDING_DIM)

# Load dataset
def load_data(filename, is_test=False):
    sentences, labels = [], []
    with open(filename, 'r', encoding='utf-8') as f:
        sentence, label = [], []
        for line in f:
            line = line.strip()
            if not line:
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                    sentence, label = [], []
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            sentence.append(parts[1])
            if not is_test:
                label.append(parts[2] if len(parts) > 2 else "O")
            else:
                label.append("O")
    if sentence:
        sentences.append(sentence)
        labels.append(label)
    return sentences, labels

# Load train, dev, and test data
train_sentences, train_labels = load_data(train_file)
dev_sentences, dev_labels = load_data(dev_file)
test_sentences, _ = load_data(test_file, is_test=True)

# Build vocabulary with GloVe embeddings
word_to_ix = {"<PAD>": 0, "<UNK>": 1}
tag_to_ix = {}

# Step 1: Create an embedding matrix
embedding_matrix = np.random.uniform(-0.1, 0.1, (len(word_to_ix) + 50000, EMBEDDING_DIM))  # Extra space for new words

for sentence in train_sentences:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

# Assign GloVe vectors to words (handling case sensitivity)
for word, idx in word_to_ix.items():
    lowercase_word = word.lower()
    if lowercase_word in glove_embeddings:
        embedding_matrix[idx] = glove_embeddings[lowercase_word]
    elif word in glove_embeddings:
        embedding_matrix[idx] = glove_embeddings[word]

# Map entity labels to indexes
for labels in train_labels:
    for tag in labels:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

ix_to_tag = {v: k for k, v in tag_to_ix.items()}

# Dataset class
class NERDataset(Dataset):
    def __init__(self, sentences, labels, word_to_ix, tag_to_ix):
        self.sentences = sentences
        self.labels = labels
        self.word_to_ix = word_to_ix
        self.tag_to_ix = tag_to_ix

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]
        word_indices = [self.word_to_ix.get(w, 1) for w in sentence]
        label_indices = [self.tag_to_ix.get(l, 0) for l in label]
        return torch.tensor(word_indices), torch.tensor(label_indices)

# Custom collate function
def collate_fn(batch):
    words, labels = zip(*batch)
    words_padded = pad_sequence(words, batch_first=True, padding_value=0)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)
    return words_padded, labels_padded

# Create datasets and loaders
train_dataset = NERDataset(train_sentences, train_labels, word_to_ix, tag_to_ix)
dev_dataset = NERDataset(dev_sentences, dev_labels, word_to_ix, tag_to_ix)
test_dataset = NERDataset(test_sentences, [["O"] * len(s) for s in test_sentences], word_to_ix, tag_to_ix)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Define BLSTM Model with GloVe Embeddings
class BLSTM_NER(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_matrix):
        super(BLSTM_NER, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=False)
        self.lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, num_layers=LSTM_LAYERS,
                            bidirectional=True, dropout=DROPOUT, batch_first=True)
        self.fc = nn.Linear(HIDDEN_DIM * 2, LINEAR_OUT_DIM)
        self.elu = nn.ELU()
        self.classifier = nn.Linear(LINEAR_OUT_DIM, tagset_size)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x)
        x = self.elu(x)
        x = self.classifier(x)
        return x

# Initialize model, loss, optimizer, and scheduler
model = BLSTM_NER(len(word_to_ix), len(tag_to_ix), embedding_matrix)
criterion = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.85, nesterov=True, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for words, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(words)
        outputs = outputs.view(-1, outputs.shape[-1])
        labels = labels.view(-1).long()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, LR: {scheduler.get_last_lr()[0]}")

# Save model
torch.save(model.state_dict(), os.path.join(output_dir, "blstm2.pt"))
print("Model saved!")

# **Save predictions as dev2 and test2**
def evaluate(model, data_loader, sentences, output_file):
    model.eval()
    predictions = []
    batch_offset = 0

    with torch.no_grad():
        for words, _ in data_loader:
            outputs = model(words)
            outputs = torch.argmax(outputs, dim=-1)

            for batch_idx, preds in enumerate(outputs):
                sentence = sentences[batch_offset]
                for word_idx, pred in enumerate(preds[:len(sentence)]):
                    predictions.append(f"{word_idx + 1} {sentence[word_idx]} {ix_to_tag[pred.item()]}\n")
                predictions.append("\n")
                batch_offset += 1

    with open(output_file, "w", encoding="utf-8") as f:
        f.writelines(predictions)

# Save outputs as dev2 and test2
evaluate(model, dev_loader, dev_sentences, os.path.join(output_dir, "dev2.out"))
evaluate(model, test_loader, test_sentences, os.path.join(output_dir, "test2.out"))

print("Training completed!")


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Epoch 1, Loss: 316.5207, LR: 0.01
Epoch 2, Loss: 232.0206, LR: 0.01
Epoch 3, Loss: 192.2139, LR: 0.01
Epoch 4, Loss: 163.6787, LR: 0.01
Epoch 5, Loss: 145.3240, LR: 0.01
Epoch 6, Loss: 131.2901, LR: 0.01
Epoch 7, Loss: 119.8484, LR: 0.01
Epoch 8, Loss: 111.0587, LR: 0.01
Epoch 9, Loss: 103.2982, LR: 0.01
Epoch 10, Loss: 96.9492, LR: 0.008
Epoch 11, Loss: 91.4502, LR: 0.008
Epoch 12, Loss: 87.2833, LR: 0.008
Epoch 13, Loss: 83.5890, LR: 0.008
Epoch 14, Loss: 80.2770, LR: 0.008
Epoch 15, Loss: 77.2407, LR: 0.008
Epoch 16, Loss: 75.0093, LR: 0.008
Epoch 17, Loss: 71.8924, LR: 0.008
Epoch 18, Loss: 69.2763, LR: 0.008
Epoch 19, Loss: 67.4815, LR: 0.008
Epoch 20, Loss: 64.9579, LR: 0.0064
Epoch 21, Loss: 63.2328, LR: 0.0064
Epoch 22, Loss: 61.8117, LR: 0.0064
Epoch 23, Loss: 60.1013, LR: 0.0064
Epoch 24, Loss: 58.8562, LR: 0.0064
Epoch 25, Loss: 57.3848, LR: 0.0064
Epoch 26, Loss: 55.9593, LR: 0.0064
Epoch 27, Loss: 55.1110, LR: 0.0064
Epoch 28, Loss: 53.8550, LR: 0.0064
Epoch 29, Loss: 52.7

In [14]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [15]:
!python /content/drive/MyDrive/hw4csci544/eval.py -p /content/drive/MyDrive/hw4csci544/dev2.out -g /content/drive/MyDrive/hw4csci544/data/dev


processed 51578 tokens with 5942 phrases; found: 5489 phrases; correct: 4428.
accuracy:  95.51%; precision:  80.67%; recall:  74.52%; FB1:  77.47
              LOC: precision:  88.42%; recall:  82.31%; FB1:  85.26  1710
             MISC: precision:  78.25%; recall:  69.85%; FB1:  73.81  823
              ORG: precision:  66.14%; recall:  64.95%; FB1:  65.54  1317
              PER: precision:  85.48%; recall:  76.06%; FB1:  80.49  1639


In [13]:
file_path = "/content/drive/MyDrive/hw4csci544/eval.py"

# Check if file exists
if os.path.exists(file_path):
    print("✅ File exists!")
else:
    print("❌ File does NOT exist!")

✅ File exists!


## Task 3

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import os
import numpy as np
import gzip

# Paths
train_file = "/content/drive/MyDrive/hw4csci544/data/train"
dev_file = "/content/drive/MyDrive/hw4csci544/data/dev"
test_file = "/content/drive/MyDrive/hw4csci544/data/test"
glove_path = "/content/drive/MyDrive/hw4csci544/glove.6B.100d.gz"
output_dir = "/content/drive/MyDrive/hw4csci544/"

# Hyperparameters
EMBEDDING_DIM = 100  # Word embedding dimension
CHAR_EMBEDDING_DIM = 30  # Character embedding dimension
HIDDEN_DIM = 256  # LSTM hidden dimension
LSTM_LAYERS = 1  # Number of LSTM layers
CNN_OUT_DIM = 50  # CNN output dimension
CNN_KERNEL_SIZE = 3  # CNN kernel size
DROPOUT = 0.1  # Dropout rate
LINEAR_OUT_DIM = 128  # Linear layer output dimension
BATCH_SIZE = 32  # Batch size
LEARNING_RATE = 0.01  # Learning rate
MOMENTUM = 0.85  # Momentum for SGD
WEIGHT_DECAY = 3e-5  # Weight decay
EPOCHS = 20  # Number of epochs

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load GloVe embeddings
def load_glove_embeddings(glove_path, embedding_dim=100):
    glove_embeddings = {}
    with gzip.open(glove_path, "rt", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            word = parts[0]
            vector = np.array(parts[1:], dtype=np.float32)
            glove_embeddings[word] = vector
    return glove_embeddings

glove_embeddings = load_glove_embeddings(glove_path, EMBEDDING_DIM)

# Load dataset
def load_data(filename, is_test=False):
    sentences, labels = [], []
    with open(filename, 'r', encoding='utf-8') as f:
        sentence, label = [], []
        for line in f:
            line = line.strip()
            if not line:
                if sentence:
                    sentences.append(sentence)
                    labels.append(label)
                    sentence, label = [], []
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            sentence.append(parts[1])
            if not is_test:
                label.append(parts[2] if len(parts) > 2 else "O")
            else:
                label.append("O")
    if sentence:
        sentences.append(sentence)
        labels.append(label)
    return sentences, labels

# Load train, dev, and test data
train_sentences, train_labels = load_data(train_file)
dev_sentences, dev_labels = load_data(dev_file)
test_sentences, _ = load_data(test_file, is_test=True)

# Build vocabulary and character vocabulary
word_to_ix = {"<PAD>": 0, "<UNK>": 1}
char_to_ix = {"<PAD>": 0, "<UNK>": 1}
tag_to_ix = {}

# Create word and character vocabularies
for sentence in train_sentences:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
        for char in word:
            if char not in char_to_ix:
                char_to_ix[char] = len(char_to_ix)

# Create embedding matrix for words
embedding_matrix = np.random.uniform(-0.1, 0.1, (len(word_to_ix) + 50000, EMBEDDING_DIM))
for word, idx in word_to_ix.items():
    lowercase_word = word.lower()
    if lowercase_word in glove_embeddings:
        embedding_matrix[idx] = glove_embeddings[lowercase_word]
    elif word in glove_embeddings:
        embedding_matrix[idx] = glove_embeddings[word]

# Map entity labels to indexes
for labels in train_labels:
    for tag in labels:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

ix_to_tag = {v: k for k, v in tag_to_ix.items()}

# Dataset class
class NERDataset(Dataset):
    def __init__(self, sentences, labels, word_to_ix, char_to_ix, tag_to_ix):
        self.sentences = sentences
        self.labels = labels
        self.word_to_ix = word_to_ix
        self.char_to_ix = char_to_ix
        self.tag_to_ix = tag_to_ix

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]
        word_indices = [self.word_to_ix.get(w, 1) for w in sentence]
        char_indices = [[self.char_to_ix.get(c, 1) for c in word] for word in sentence]
        label_indices = [self.tag_to_ix.get(l, 0) for l in label]
        return torch.tensor(word_indices), char_indices, torch.tensor(label_indices)

# Custom collate function
def collate_fn(batch):
    words, chars, labels = zip(*batch)

    # Pad word sequences
    words_padded = pad_sequence(words, batch_first=True, padding_value=0)

    # Pad label sequences
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-1)

    # Pad character sequences
    max_word_len = max([max([len(word) for word in sentence]) for sentence in chars])  # Longest word in the batch
    max_sentence_len = max([len(sentence) for sentence in chars])  # Longest sentence in the batch

    # Initialize a tensor for padded character sequences
    chars_padded = torch.zeros((len(chars), max_sentence_len, max_word_len), dtype=torch.long)

    # Fill the tensor with character indices
    for i, sentence in enumerate(chars):
        for j, word in enumerate(sentence):
            chars_padded[i, j, :len(word)] = torch.tensor(word, dtype=torch.long)

    return words_padded, chars_padded, labels_padded

# Create datasets and loaders
train_dataset = NERDataset(train_sentences, train_labels, word_to_ix, char_to_ix, tag_to_ix)
dev_dataset = NERDataset(dev_sentences, dev_labels, word_to_ix, char_to_ix, tag_to_ix)
test_dataset = NERDataset(test_sentences, [["O"] * len(s) for s in test_sentences], word_to_ix, char_to_ix, tag_to_ix)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Define LSTM-CNN Model
class LSTM_CNN_NER(nn.Module):
    def __init__(self, vocab_size, char_vocab_size, tagset_size, embedding_matrix, char_embedding_dim=30):
        super(LSTM_CNN_NER, self).__init__()
        # Word embeddings
        self.word_embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=False)
        # Character embeddings
        self.char_embedding = nn.Embedding(char_vocab_size, char_embedding_dim, padding_idx=0)
        # CNN for character-level features
        self.cnn = nn.Conv1d(char_embedding_dim, CNN_OUT_DIM, CNN_KERNEL_SIZE, padding=1)
        # BLSTM for word-level features
        self.lstm = nn.LSTM(EMBEDDING_DIM + CNN_OUT_DIM, HIDDEN_DIM, num_layers=LSTM_LAYERS,
                            bidirectional=True, dropout=DROPOUT, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(HIDDEN_DIM * 2, LINEAR_OUT_DIM)
        self.elu = nn.ELU()
        # Output layer
        self.classifier = nn.Linear(LINEAR_OUT_DIM, tagset_size)

    def forward(self, words, chars):
        # Word embeddings
        word_embeds = self.word_embedding(words)
        # Character embeddings
        batch_size, seq_len, word_len = chars.size()
        chars = chars.view(batch_size * seq_len, word_len)
        char_embeds = self.char_embedding(chars)
        char_embeds = char_embeds.permute(0, 2, 1)
        # CNN for character-level features
        char_features = self.cnn(char_embeds)
        char_features, _ = torch.max(char_features, dim=2)
        char_features = char_features.view(batch_size, seq_len, -1)
        # Concatenate word and character features
        combined_embeds = torch.cat((word_embeds, char_features), dim=2)
        # BLSTM
        lstm_out, _ = self.lstm(combined_embeds)
        # Fully connected layer
        fc_out = self.fc(lstm_out)
        fc_out = self.elu(fc_out)
        # Output layer
        logits = self.classifier(fc_out)
        return logits

# Initialize model, loss, optimizer, and scheduler
model = LSTM_CNN_NER(len(word_to_ix), len(char_to_ix), len(tag_to_ix), embedding_matrix)
criterion = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, nesterov=True, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for words, chars, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(words, chars)
        outputs = outputs.view(-1, outputs.shape[-1])
        labels = labels.view(-1).long()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, LR: {scheduler.get_last_lr()[0]}")

# Save model
torch.save(model.state_dict(), os.path.join(output_dir, "blstm3.pt"))
print("Model saved!")

# Evaluation function
def evaluate(model, data_loader, sentences, output_file):
    model.eval()
    predictions = []
    batch_offset = 0

    with torch.no_grad():
        for words, chars, _ in data_loader:
            outputs = model(words, chars)
            outputs = torch.argmax(outputs, dim=-1)

            for batch_idx, preds in enumerate(outputs):
                sentence = sentences[batch_offset]
                for word_idx, pred in enumerate(preds[:len(sentence)]):
                    predictions.append(f"{word_idx + 1} {sentence[word_idx]} {ix_to_tag[pred.item()]}\n")
                predictions.append("\n")
                batch_offset += 1

    with open(output_file, "w", encoding="utf-8") as f:
        f.writelines(predictions)

# Save outputs as dev3.out and test3.out
evaluate(model, dev_loader, dev_sentences, os.path.join(output_dir, "dev3.out"))
evaluate(model, test_loader, test_sentences, os.path.join(output_dir, "test3.out"))

print("Training completed!")

Epoch 1, Loss: 308.0103, LR: 0.01
Epoch 2, Loss: 202.5471, LR: 0.01
Epoch 3, Loss: 146.6911, LR: 0.01
Epoch 4, Loss: 111.8269, LR: 0.01
Epoch 5, Loss: 88.8739, LR: 0.01
Epoch 6, Loss: 71.9349, LR: 0.01
Epoch 7, Loss: 59.0237, LR: 0.01
Epoch 8, Loss: 49.9647, LR: 0.01
Epoch 9, Loss: 44.6148, LR: 0.01
Epoch 10, Loss: 40.6931, LR: 0.008
Model saved!
Training completed!


In [5]:
!python /content/drive/MyDrive/hw4csci544/eval.py -p /content/drive/MyDrive/hw4csci544/dev3.out -g /content/drive/MyDrive/hw4csci544/data/dev


processed 51578 tokens with 5942 phrases; found: 6037 phrases; correct: 4674.
accuracy:  96.49%; precision:  77.42%; recall:  78.66%; FB1:  78.04
              LOC: precision:  88.55%; recall:  81.27%; FB1:  84.76  1686
             MISC: precision:  59.09%; recall:  72.23%; FB1:  65.01  1127
              ORG: precision:  66.11%; recall:  70.69%; FB1:  68.32  1434
              PER: precision:  87.54%; recall:  85.07%; FB1:  86.29  1790
